# Molecular Dynamics
This Jupyter Notebook provides a step-by-step guide for performing a Molecular Dynamics (MD) simulation using GROMACS. Initially, dependencies such as GROMACS and Python libraries are installed. Subsequently, the initial molecular structure is preprocessed, solvated, and various simulation parameters are set up. Multiple stages of the MD simulation, including energy minimization, equilibration, and production MD, are then executed. Lastly, trajectory analysis is performed, including the calculation of parameters like RMSD (Root Mean Square Deviation) and gyration, offering insights into the molecular system's behavior over time. This notebook is employed as a comprehensive tool for simulating and analyzing the molecular dynamics of the specified system.

## Input/Output Description
- **Input:**
1. A PDB file (`maws_pfoa_aptamer.pdb`) representing the initial molecular structure.
1. GROMACS parameter and configuration files (`ions.mdp`, `minim.mdp`, `nvt.mdp`, `npt.mdp`, `md.mdp`) for different simulation stages.

`ions.mdp`: parameters for simulating the addition of ions to the system to achieve a desired salt concentration.

`minim.mdp`: energy minimization, which aims to minimize the energy of the system to achieve a stable starting point for simulations.

`nvt.mdp`: parameters for an NVT ensemble simulation, where the number of particles, volume, and temperature of the system are held constant.

`npt.mdp`: NPT ensemble simulations, where the system is equilibrated at constant temperature and pressure.

`md.mdp`: main production phase of the MD simulation, specifying simulation parameters for data collection on the system's dynamics.

- **Output:**
1. Processed and solvated molecular structures in GROMACS formats.
1. Energy, temperature, pressure, and density profiles during the simulation.
1. Trajectory and analysis files including RMSD, gyration, and more.

### REMEMBER: change the Jupyter kernel to **`Python (AptaLoop)`**. See our README file for detailed information.


In [ ]:
%%bash
pip install --upgrade pip
pip install wheel
pip install setuptools
pip install --upgrade setuptools wheel
pip install --upgrade cmake pybind11
pip install nglview
pip install GromacsWrapper

  Obtaining dependency information for cmake from https://files.pythonhosted.org/packages/72/89/b1cf3cd5fb9f4ae796dd4a743412553f884dad2acbf6b9828d3a0c2b5524/cmake-3.27.6-py2.py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata
  Obtaining dependency information for pybind11 from https://files.pythonhosted.org/packages/06/55/9f73c32dda93fa4f539fafa268f9504e83c489f460c380371d94296126cd/pybind11-2.11.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.0/26.0 MB 9.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 5.4 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 11.8 MB/s eta 0:00:0000:0100:01
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.

In [2]:
import subprocess
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np 
import gromacs 
import gromacs.formats
import nglview as nv
import os

NOTE: Some configuration directories are not set up yet: 
	/home/jupyter/.gromacswrapper
	/home/jupyter/.gromacswrapper/qscripts
	/home/jupyter/.gromacswrapper/templates
NOTE: You can create the configuration file and directories with:
	>>> import gromacs
	>>> gromacs.config.setup()


### Replace `maws_pfoa_aptamer.pdb` with the path of your file (relative to the script path) in the following cell

In [3]:
script_path = os.getcwd()
input_file = os.path.join(script_path, "maws_pfoa_aptamer.pdb")

First of all, we need to check if the file has a compatible format. We are specifically checking if the Residue Name column (4th column) of the PDB input file starts with either `D` or `R`. If it doesn't, please edit the file before continuing with the NB.

In [5]:
def check_file_format(infile):
    with open(infile, 'r') as file:
        for line in file:
            if line.startswith(("TER", "END", "CONECT")) or ("LIG" in line.split()[3:4]):
                continue
            parts = line.split()
            if len(parts) >= 4:
                column = parts[3]
                if column.endswith(('A', 'T', 'G', 'C', 'U', '5', '3')) and (column.startswith('D') or column.startswith('R')):
                    continue
                else:
                    return "The file does not have the correct format. Edit the file so that the 4th column starts always with either 'D' or 'R'."
    return "The file has the correct format, you may proceed :)"
print(check_file_format(input_file))

The file has the correct format, you may proceed :)


Here, the analysis using GROMACS starts. First, it removes all lines that contain 'HOH' (the PDB code for water) from the file whose path is stored in the variable `input_file`. The resulting file's content is then written to `python_aptamer.pdb`.

In [12]:
!grep -v HOH $input_file > python_aptamer.pdb

Then, converts a PDB file to a GROMACS-ready structure file (`.gro`), atom types, and topology while ignoring any hydrogen atoms that are defined in the PDB file. It uses the Amber03 force field and TIP3P water model.

In [13]:
!gmx pdb2gmx -f python_aptamer.pdb -o python_aptamer_processed.gro -ignh -ff amber03 -water tip3p

                     :-) GROMACS - gmx pdb2gmx, 2021.4 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Mu

Next, edits the GROMACS structure file to define a cubic box around the molecule. The box would start 0.5nm away from the molecule in all directions.

In [14]:
!gmx editconf -f python_aptamer_processed.gro -o python_newbox.gro -c -d 0.5 -bt cubic

                     :-) GROMACS - gmx editconf, 2021.4 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu M

Solvates (i.e., adds water to) the system using a pre-equilibrated box of SPC water. Modifies the topology file `topol.top` accordingly to include the added water molecules.

In [15]:
!gmx solvate -cp python_newbox.gro -cs spc216.gro -o python_solv.gro -p topol.top

                     :-) GROMACS - gmx solvate, 2021.4 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Mu

Pre-processes the system for the ion-adding stage by creating a binary run input file that contains the data needed for the next `mdrun` command.

In [16]:
!gmx grompp -f ions.mdp -c python_solv.gro -p topol.top -o ions.tpr

                      :-) GROMACS - gmx grompp, 2021.4 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Mu

Adds ions to the system to make the total charge zero. Replaces water molecules by sodium and/or chloride ions, updates the topology file `topol.top` accordingly.

In [17]:
!printf "SOL" | gmx genion -s ions.tpr -o python_solv_ions.gro -p topol.top -pname NA -nname CL -neutral

                      :-) GROMACS - gmx genion, 2021.4 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Mu

Pre-processes the system for the energy minimization stage.

In [18]:
!gmx grompp -f minim.mdp -c python_solv_ions.gro -p topol.top -o em.tpr

                      :-) GROMACS - gmx grompp, 2021.4 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Mu

Runs the energy minimization.

In [19]:
!gmx mdrun -v -deffnm em

                      :-) GROMACS - gmx mdrun, 2021.4 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Mur

Extracts the potential energy information from the energy file and write it to `potential.xvg`.

In [20]:
!printf "10 0"| gmx energy -f em.edr -o potential.xvg

                      :-) GROMACS - gmx energy, 2021.4 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Mu

Pre-processes the system for the temperature equilibration (NVT) stage.

In [25]:
!gmx grompp -f nvt.mdp -c em.gro -r em.gro -p topol.top -o nvt.tpr

                      :-) GROMACS - gmx grompp, 2021.4 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Mu

Runs constant number of particles, volume, and temperature (NVT) simulation.

In [26]:
!gmx mdrun -deffnm nvt

                      :-) GROMACS - gmx mdrun, 2021.4 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Mur

Extracts the temperature data from the energy file and write it to `temperature.xvg`.

In [27]:
!printf "16 0" | gmx energy -f nvt.edr -o temperature.xvg

                      :-) GROMACS - gmx energy, 2021.4 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Mu

Pre-processes the system for the pressure equilibration (NPT) stage.

In [28]:
!gmx grompp -f npt.mdp -c nvt.gro -r nvt.gro -t nvt.cpt -p topol.top -o npt.tpr

                      :-) GROMACS - gmx grompp, 2021.4 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Mu

Runs constant number of particles, pressure, and temperature (NPT) simulation.

In [29]:
!gmx mdrun -deffnm npt

                      :-) GROMACS - gmx mdrun, 2021.4 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Mur

Extracts the pressure data from the energy file and write it to `pressure.xvg`.

In [30]:
!printf "18 0" | gmx energy -f npt.edr -o pressure.xvg

                      :-) GROMACS - gmx energy, 2021.4 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Mu

Extracts the density data from the energy file and write it to `density.xvg`.

In [31]:
!printf "24 0" | gmx energy -f npt.edr -o density.xvg

                      :-) GROMACS - gmx energy, 2021.4 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Mu

Pre-processes the system for the production MD run stage.

In [32]:
!gmx grompp -f md.mdp -c npt.gro -t npt.cpt -p topol.top -o md_python.tpr

                      :-) GROMACS - gmx grompp, 2021.4 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Mu

Runs the production molecular dynamics.

In [33]:
!gmx mdrun -deffnm md_python # Use the option "-cpi md_python.cpt" to resume analysis

                      :-) GROMACS - gmx mdrun, 2021.4 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Mur

Converting the trajectory file (`md_python.xtc`) to the one without periodic boundary conditions (`md_python_noPBC.xtc`). It's also centering the molecule in the box.

In [34]:
!printf "1 0" | gmx trjconv -s md_python.tpr -f md_python.xtc -o md_python_noPBC.xtc -pbc mol -center

                     :-) GROMACS - gmx trjconv, 2021.4 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Mu

Calculates the root mean square deviation (RMSD) of the trajectory, with respect to the structure in `md_0_1.tpr`, which is probably the initial structure but after equilibration. The output is written to `rmsd.xvg`.

In [35]:
!printf "4 0" | gmx rms -s md_0_1.tpr -f md_python_noPBC.xtc -o rmsd.xvg -tu ns

                       :-) GROMACS - gmx rms, 2021.4 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Murt

Calculates the RMSD of the trajectory with respect to the structure obtained after energy minimization. The output is written to `rmsd_xtal.xvg`.

In [36]:
!printf "1 0" | gmx rms -s em.tpr -f md_python_noPBC.xtc -o rmsd_xtal.xvg -tu ns

                       :-) GROMACS - gmx rms, 2021.4 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Murt

Finally, calculates the radius of gyration (`Rg`) of the molecule throughout the trajectory. The output is written to `gyrate.xvg`.

In [37]:
!printf "1 0" | gmx gyrate -s md_python.tpr -f md_python_noPBC.xtc -o gyrate.xvg

                      :-) GROMACS - gmx gyrate, 2021.4 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Mu